In [1]:
%load_ext jupyter_black

<div style="display: flex; align-items: center;">
  <div style="flex: 1;">
    <img src="images/notebook.gif" alt="segment" width="500">
  </div>
  <div style="flex: 1;">
    <h2> Practical Prompt Engineering </h2>
    <br>
    <div>
       In this tutorial, you’ll learn how to:
       <ul>
        <li>Apply prompt engineering techniques to practical, real-world examples
        <li>Tap into the power of roles in messages to go beyond using singular role prompts
        <li>Use numbered steps, delimiters, few-shot prompting and other techniques to improve your results
        <li>Understand and use chain-of-thought prompting to add more context
      </ul>
    <div>

  </div>
</div>

First let's import all the libraries we're going to use...

In [2]:
import os
import openai

and retrieve our OpenAI API key...

In [64]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

The next function allows us to send prompts to the model and get a response back. <br>

We have specified the following model parameters:
* Model: **GPT-3.5-turbo**
* Temperature: **0**

The function takes as inputs <u>prompt</u> and a <u>list of previous messages</u>, and returns a new list of messsages that include the prompt and the response from the model:

In [ ]:
def get_completion_from_messages(
    prompt,
    messages,
    model="gpt-3.5-turbo",
    temperature=0
):

    if not isinstance(messages, list):
        messages = [messages]

    messages.append(prompt)
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    print(response.choices[0].message["content"])

    messages.append(response.choices[0].message)
    return messages

Let's test it!

In [ ]:
messages = [
    {"role": "user", "content": "My name is Alex"},
]

prompt = {"role": "user", "content": "What is my name?"}

response = get_completion_from_messages(prompt, messages)

In [41]:
def fprint(output):
    if isinstance(output, list):
        for item in output:
            if "role" in item and "content" in item:
                role = item["role"]
                content = item["content"].strip()

                if role == "user":
                    prefix = "User"
                elif role == "assistant":
                    prefix = "Assistant"
                    content = content.replace(
                        "\n\n", "\n"
                    )  # Replace double newlines with single newlines
                else:
                    prefix = role.capitalize()

                formatted_item = f"{prefix}: \n {content}\n"
                print(formatted_item)
                print("---" * 30)
            else:
                print("Invalid output format")
    else:
        print("Invalid output format")

# Ways to Engineer your Prompts

We will now look at a few different examples of how you can improve your promtps to become the utlimate prompt engineer!

First, we have a list of reviews written below. Our task is to summarise them using ChatGPT. <br> Let's see what we can do:

In [37]:
reviews = [
    (
        "Date: December 15, 2021; Username: John123; Review: I am absolutely delighted \
    with this savings product! The interest rates are fantastic, and it has helped \
    me grow my savings significantly. Highly recommend!"
    ),
    (
        "Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite \
    disappointed with this savings product. The promised returns were not as \
    impressive as advertised, and the fees associated with it added up quickly. \
    Not worth it."
    ),
    (
        "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather \
    indifferent about this savings product. It's just like any other basic savings \
    account out there. Nothing special, but it does the job."
    ),
]

In [76]:
content = f"""Summarise these 3 reviews:{reviews}"""

print(content)

Summarise these 3 reviews:['Date: December 15, 2021; Username: John123; Review: I am absolutely delighted     with this savings product! The interest rates are fantastic, and it has helped     me grow my savings significantly. Highly recommend!', 'Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite     disappointed with this savings product. The promised returns were not as     impressive as advertised, and the fees associated with it added up quickly.     Not worth it.', "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather     indifferent about this savings product. It's just like any other basic savings     account out there. Nothing special, but it does the job."]


In [48]:
prompt = {
    "role": "user",
    "content": content,
}

messages = get_completion_from_messages(prompt, [])

Review 1: John123 is extremely satisfied with the savings product, praising the fantastic interest rates and significant growth of their savings. Highly recommends it.

Review 2: Sarah77 is disappointed with the savings product, as the promised returns were not as impressive as advertised and the associated fees accumulated quickly. Considers it not worth it.

Review 3: AlexSmith expresses indifference towards the savings product, considering it similar to any other basic savings account. It is deemed as nothing special but gets the job done.


Great! It provided a summary of each review. But it's safe to say this is by no means useful enough to be a product yet. 

Let's see what we can do with some simple prompt engineering.

## System Prompt

The system message is a behind-the-scenes prompt sent to the model before the user begins querying. <br>
Typically we can use it to determine what **role** the AI should play and how it should behave generally. <br> 
For example, if you are creating a Pizza Ordering Chatbot, then you can specify that in the system message.

- **System role:** Allows you to specify the way the model answers questions. Classic example: “You are a helpful assistant.”

- **User role:** Equivalent to the queries made by the user.

- **Assistant role:** are the model’s responses (based on the user messages)


### Role Prompting

In this example we are setting the **role** of the model via the system message. This is known as "Role Prompting", and is a general practice to help to set the tone and context for the model's responses.

In [60]:
system_message = {
    "role": "system",
    "content": (
        """
        You are a Review Summariser. \
        Your job is to process reviews from customers, \
        and summarise them for analysis for the customer review team.
        """
    ),
}

In [53]:
prompt = {
    "role": "user",
    "content": f"""Summarise these 5 reviews:

    {reviews}
    """,
}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

Review 1: John123 is delighted with the savings product, praising the fantastic interest rates and significant growth of their savings. Highly recommends.

Review 2: Sarah77 is disappointed with the savings product. The promised returns were not as impressive as advertised, and the fees added up quickly. Not worth it.

Review 3: AlexSmith is indifferent about the savings product. It is described as just like any other basic savings account, nothing special, but it does the job.


In [61]:
fprint(messages)

System: 
 You are a Review Summariser.         Your job is to process reviews from customers,         and summarise them for analysis for the customer review team.

------------------------------------------------------------------------------------------
User: 
 Summarise these 5 reviews:

    ['Date: December 15, 2021; Username: John123; Review: I am absolutely delighted     with this savings product! The interest rates are fantastic, and it has helped     me grow my savings significantly. Highly recommend!', 'Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite     disappointed with this savings product. The promised returns were not as     impressive as advertised, and the fees associated with it added up quickly.     Not worth it.', "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather     indifferent about this savings product. It's just like any other basic savings     account out there. Nothing special, but it does the job."]

-------